<a href="https://colab.research.google.com/github/abjilox/stablediffusion_colab/blob/main/stable_diffusion_ui.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Stable Diffusion for colab by abjilox

### This implements [AUTOMATIC1111 Stable Diffusion Web UI](https://github.com/AUTOMATIC1111/stable-diffusion-webui) in colab

Latest version of this notebook can be found at [here](https://colab.research.google.com/github/abjilox/stablediffusion_colab/blob/main/stable_diffusion_ui.ipynb).


ngrok can be used optional





In [ ]:
import os

class SDWebUI:
    class Config:
        def __init__(self):
            self.use_google_drive = False #@param {type:"boolean"}
            self.share_gradio = True #@param {type:"boolean"}
            self.tag = 'v1.4.0' #@param {type:"string"}
            self.username = 'a' #@param {type:"string"}
            self.password = 'a' #@param {type:"string"}
            self.ngrok_token = '' #@param {type: "string"}
            self.repo_dir = 'SDWEBUI' #@param {type: "string"}
            self.tag_file = f'{self.repo_dir}/tag.txt'
            self.workdir = '/content'

    class Args:
        def __init__(self, setup):
            self.setup = setup
            self.config = setup.config
            self.logger = setup.logger

        def args(self):
            args = ['--xformers', 
                    '--gradio-img2img-tool color-sketch', 
                    '--enable-insecure-extension-access',
                    '--gradio-queue']
            args.append(self.ngrok_args())
            args.append(self.share_args())
            args.append(self.gradio_auth_args())
            return ' '.join(args)

        def ngrok_args(self):
            if self.config.ngrok_token != '':
                self.log('Using ngrok')
                return f'--ngrok {self.config.ngrok_token}'
            else:
                return ''
        
        def share_args(self):
            if self.config.share_gradio and self.config.ngrok_token == '':
                self.log('Sharing on Gradio')
                return '--share'
            else:
                self.log('Not sharing on Gradio')
                return ''
            
        def gradio_auth_args(self):
            if self.config.username != '' and self.config.password != '':
                self.log('Using Gradio auth')
                return f'--gradio-auth {self.config.username}:{self.config.password}'
            else:
                self.log('Not using Gradio auth')
                return ''
            
        def log(self, message):
            self.logger.log(f'Args - {message}')     
        
    class Logger:
        def __init__(self):
            pass

        def log(self, message):
            print(message)
        
    class Setup:
        def __init__(self):
            self.logger = SDWebUI.Logger()
            self.config = SDWebUI.Config()
            self.args = SDWebUI.Args(self)

        def setup(self):
            self.log('Setting up...')
            self.init_google_drive()
            %cd {self.config.workdir}
            self.clone_repo_if_needed()
            %cd {self.config.repo_dir}
            self.log(f'Directory: {os.getcwd()}')
            self.log('Loading requirements...')
            %pip install -q -r requirements.txt
            %pip install -q xformers
            self.log('Loaded')

        def download_webui(self):
            self.log(f'Downloading webui tag {self.config.tag} into {self.config.repo_dir}...')
            !curl https://github.com/AUTOMATIC1111/stable-diffusion-webui/archive/refs/tags/{self.config.tag}.zip -L -s -o stable-diffusion-webui.zip
            !unzip -q stable-diffusion-webui.zip
            !rm -rf {self.config.repo_dir}
            !mv stable-diffusion-webui-{self.config.tag} {self.config.repo_dir}
            !echo {self.config.tag} > {self.config.tag_file}
            !rm text-generation-webui.zip
            self.log('Downloaded')

        def init_google_drive(self):
            if self.config.use_google_drive:
                self.log('Mounting Google Drive...')
                from google.colab import drive
                drive.mount(f'{self.config.workdir}/drive')
                self.config.workdir = f'{self.config.workdir}/drive/MyDrive'
                self.log(f'Mounted on: {self.config.workdir}')

        def clone_repo_if_needed(self):
            if os.path.isdir(self.config.repo_dir):
                old_tag = open(self.config.tag_file, 'r').read()
                if old_tag != self.config.tag:
                    self.download_webui()
                else:
                    self.log('Repo already up to date')
            else:
                self.download_webui()   

        def init_ngrok(self):
            if self.config.ngrok_token != '':
                self.log('Installing ngrok...')
                %pip install pyngrok

        def log(self, message):
            self.logger.log(f'Setup - {message}')        

    @classmethod
    def start_ui(cls):
        setup = cls.Setup()
        args = setup.args
        setup.setup()
        setup.logger.log('Starting server...')
        !python launch.py {args.args()}

SDWebUI.start_ui()